In [15]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

In [7]:
llm=Ollama(
    model='gemma2:27b'
    )
embeddings = OllamaEmbeddings(model='bge-m3')

C:\Users\chen\AppData\Local\Temp\ipykernel_30912\264499729.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(
C:\Users\chen\AppData\Local\Temp\ipykernel_30912\264499729.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model='bge-m3')


In [8]:
print(llm)

Ollama
Params: {'model': 'gemma2:27b', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}


In [26]:
# 載入資料夾中所有TXT檔案
loader = DirectoryLoader('./', glob='**/*.txt')

# 將資料轉成document物佚，每個檔案會為作為一個document
docs = loader.load()
len(docs)
 

The MIME type of '消費爭議調解辦法 (行政院全球資訊網-主管法規).txt' is "cannot open `\\346\\266\\210\\350\\262\\273\\347\\210\\255\\350\\255\\260\\350\\252\\277\\350\\247\\243\\350\\276\\246\\346\\263\\225 (\\350\\241\\214\\346\\224\\277\\351\\231\\242\\345\\205\\250\\347\\220\\203\\350\\263\\207\\350\\250\\212\\347\\266\\262-\\344\\270\\273\\347\\256\\241\\346\\263\\225\\350\\246\\217).txt' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of '消費者保護法.txt' is "cannot open `\\346\\266\\210\\350\\262\\273\\350\\200\\205\\344\\277\\235\\350\\255\\267\\346\\263\\225.txt' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of '消費者保護細則.txt' is "cannot open `\\346\\266\\210\\350\\262\\273\\350\\200\\205\\344\\277\\235\\350\\255\\267\\347\\264\\260\\345\\211\\207.txt' (illegal byte sequence)". This file type is not currently supported in unstructured.


3

In [ ]:
docs[0]


3

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [28]:
vectordb = Chroma.from_documents(docs, embeddings)
# 將向量資料庫設為檢索器
retriever = vectordb.as_retriever()

In [33]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt_answer=ChatPromptTemplate.from_template("""
    你是一問中文專家，請用繁體中文回答問題，不知道答案就說不知道。
    參考資料:{context}
    問題:{question}
    回答:
""")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

c:\Users\chen\anaconda3\envs\langchainragas\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # | prompt
    | prompt_answer
    | llm
    | StrOutputParser()
)

In [41]:
questions = [
"為預防消費糾紛,保護消費者權益,中央主管機關得選擇特定行業,擬訂其定型化契約應記載或不得記載事項,報請行政院核定後公告,下列敘述何者錯誤?(A)公告不得記載之事項,經記載於定型化契約,仍構成契約之內容(B)違反公告之定型化契約,其定型化契約條款無效(C)公告應記載之事項,雖未記載於定型化契約,仍構成契約之內容(D)企業經營者使用定型化契約者,主管機關得隨時派員查核",
"依消費者保護法規定,下列關於企業經營者提供商品之敘述,何者正確?(A)對於有危害消費者財產之虞的商品,僅負於明顯處為警告標示之義務(B)對於第三人的損害賠償責任,得預先約定免除(C)對於消費者的損害賠償責任,得預先約定限制(D)對於有事實足認有危害消費者安全與健康之虞的商品,應即回收",
"依消費者保護法規定,企業經營者以定型化契約條款使消費者拋棄消費者審閱之合理期間,其契約條款之效力如何?(A)消費者不得主張該條款仍構成契約之內容(B)其條款不構成契約之內容(C)由行政院消費者保護處審定(D)由目的事業主管機關得撤銷",
"依消費者保護法規定,下列有關消費者保護團體之敘述,何者錯誤?(A)消費者保護團體以社團法人或財團法人為限(B)以保護消費者權益、推行消費者教育為宗旨(C)消費者保護刊物之編印發行為其任務之一(D)以確保公平競爭、促進經濟之安定與繁榮為宗旨",
"依消費者保護法規定關於商品標示,下列敘述何者錯誤?(A)企業經營者應依商品標示法等法令為商品之標示(B)輸入之商品,應附中文標示及說明書(C)輸入之商品,所附中文標示及說明書,其內容得就原產地之標示及說明,簡略摘要之(D)輸入之商品或服務在原產地附有警告標示者,應附中文標示及說明書,其內容不得較原產地之標示及說明書簡略",
"依消費者保護法規定,企業經營者所出具之書面保證書其應載明事項,下列何者錯誤?(A)交易日期(B)製造商之資本額(C)保證之內容(D)製造商之名稱、地址",
"依消費者保護法規定,下列有關消費者保護團體從事檢驗之敘述,何者錯誤?(A)得委託設有與檢驗項目有關之檢驗設備之機關、團體檢驗(B)檢驗結果有錯誤時,應主動對外更正(C)發表檢驗結果後,通知相關企業經營者,得不公布其檢驗方法及經過(D)為從事商品或服務檢驗,應設置與檢驗項目有關之檢驗設備",
"依消費者保護法規定,有關消費關係之定義,下列敘述何者正確?(A)指消費者與消費者間就商品所發生之法律關係(B)指企業經營者與企業就商品製作所發生之法律關係(C)指企業經營者與企業經營者間就商品所發生之法律關係(D)指消費者與企業經營者間就商品或服務所發生之法律關係",
"依消費者保護法規定,下列關於定型化契約條款之敘述,何者錯誤?(A)定型化契約條款未經記載於定型化契約中而依正常情形顯非消費者所得預見者,該條款不構成契約之內容(B)契約中之定型化契約條款牴觸個別磋商條款之約定者,其牴觸部分無效(C)定型化契約中之定型化契約條款,不構成契約內容之一部者,除去該部分,該契約之其他部分,仍為有效(D)定型化契約中之定型化契約條款,一部無效,該契約全部無效",
"依消費者保護法規定,提供之商品或服務具有危害消費者健康、財產之可能者,下列何者應於明顯處為警告標示及緊急處理危險之方法?(A)企業經營者(B)目的事業主管機關(C)行政院消費者保護處(D)消費者",
"依消費者保護法規定,下列關於定型化契約之敘述,何者正確?(A)中央主管機關不得選擇特定行業,公告定型化契約之審閱期間(B)以定型化契約條款使消費者拋棄合理審閱全部條款期間之權利者,無效(C)訂立定型化契約前,應有10日內之合理期間,供消費者審閱全部條款內容(D)定型化契約條款如有疑義時,應為有利於企業經營者之解釋",
"依消費者保護法規定,未經消費者要約而對之郵寄或投遞之商品,有關消費者之義務,下列敘述何者正確?(A)負通知之義務(B)負檢視之義務(C)不負保管之義務(D)負寄回之義務",
"依消費者保護法規定,下列何者非屬企業經營者以通訊交易方式訂立契約時,應將其資訊以清楚易懂之文句記載於書面,提供消費者之事項?(A)商品或服務之內容、對價(B)企業經營者之名稱(C)產品責任險之金額(D)付款期日及方式",
"依消費者保護法規定,下列有關刊登或報導廣告媒體經營者之敘述,何者錯誤?(A)由媒體經營者負連帶責任之損害賠償,得預先約定限制(B)明知廣告內容與事實不符者,就消費者因信賴該廣告所受損害,與企業經營者負連帶責任(C)由媒體經營者負連帶責任之損害賠償,不得預先約定限制(D)可得而知廣告內容與事實不符者,就消費者因信賴該廣告所受損害,與企業經營者負連帶責任",
"依消費者保護法規定,企業經營者與消費者分期付款買賣契約應以書面為之,下列何者非屬其契約書應載明事項?(A)利率(B)各期價款與其他附加費用合計之總價款與現金交易價格之差額(C)保障期間及其起算方法(D)頭期款",
"依消費者保護法規定,訪問交易之消費者以退回商品方式解除契約時,下列敘述何者錯誤?(A)無須說明理由(B)無須負擔任何費用(C)無須負擔任何對價(D)約定須在3日內退回商品",
"依消費者保護法規定,消費者於收受商品後在法定期間內,有以下何種行為契約視為解除?(A)發出書面者(B)託人轉知者(C)電話通知者(D)口頭通知者",
"依消費者保護法規定,企業經營者對於提供之商品或服務具有危害消費者健康、財產之可能者,未於明顯處為警告標示及緊急處理危險之方法,致生損害於消費者或第三人時,下列何者應負連帶賠償責任?(A)目的事業主管機關(B)行政院消費者保護處(C)企業經營者(D)消費者",
"下列何事項非屬政府為達成制定消費者保護法之目的,應就與之有關之法規及其執行情形,定期檢討、協調、改進之?(A)確保商品或服務之廣告,符合法令規定(B)促進商品或服務之公平交易(C)扶植、獎助企業經營者(D)確保商品或服務之度量衡,符合法令規定",
"消費者保護團體,須經許可設立至少幾年以上,置有消費者保護專門人員,且申請行政院評定優良者,得以自己之名義,依消費者保護法規定提起消費者損害賠償訴訟或不作為訴訟?(A)3年(B)2年(C)1年(D)4年",
"依消費者保護法規定,下列有關訪問交易之敘述,何者錯誤?(A)經營者未經邀約而與消費者在其工作場所所訂立之契約(B)經營者未經邀約而與消費者在公共場所所訂立之契約(C)經營者未經邀約而與消費者在其住居所所訂立之契約(D)經營者與消費者邀約而在其住居所所訂立之契約",
"依消費者保護法規定,定型化契約中之條款,下列何者非屬得推定其顯失公平之情形?(A)契約之主要權利或義務,因受條款之限制,致契約之目的難以達成者(B)條款之全部或一部無效者(C)違反平等互惠原則者(D)條款與其所排除不予適用之任意規定之立法意旨顯相矛盾者",
"依消費者保護法規定,下列有關消費爭議調解之敘述,何者錯誤?(A)列席協同調解人對於調解事件之內容,除已公開之事項外,應保守秘密(B)調解事件之內容應保守秘密,絕對不得公開(C)消費爭議調解委員會之調解程序,得不公開(D)調解程序,於直轄市、縣(市)政府或其他適當之處所行之",
"依要約書定型化契約應記載及不得記載事項,下列何者屬不得記載事項之規定?(A)約定承購總價款、付款條件及其他要約條件(B)約定在契約成立前,受託人得向消費者收取斡旋金、訂金或其他任何名目之費用(C)簽訂不動產買賣契約書之期間(D)不動產買賣標的",
"依消費者保護法規定,下列有關企業經營者保證書之敘述,何者錯誤?(A)保證書應載明保證期間及其起算方法(B)對消費者保證商品之品質時,應主動出具書面保證書(C)保證書應載明交易日期、保證之內容(D)保證書應載明保證金額",
"依消費者保護法規定,從事經銷之企業經營者,就商品或服務所生之損害,下列敘述何者錯誤?(A)與提供服務之企業經營者,負全部賠償責任(B)對於損害之防免,已盡相當之注意而仍不免發生損害者,不在此限(C)與提供服務之企業經營者,連帶負賠償責任(D)對於損害之防免,縱加以相當之注意而仍不免發生損害者,不在此限",
"依消費者保護法規定,有關消費爭議之定義,下列敘述何者正確?(A)指企業經營者與企業就商品製作所發生之爭議(B)指消費者與消費者間就商品所發生之爭議(C)指消費者與企業經營者間就商品或服務所發生之爭議(D)指企業經營者與企業經營者間就商品所發生之爭議",
"依消費者保護法規定,有關消費訴訟之定義,下列敘述何者正確?(A)指企業經營者與企業經營者間就商品所發生之訴訟(B)指企業經營者與企業就商品製作所發生之訴訟(C)指因消費關係而向法院提起之訴訟(D)指消費者與消費者間就商品所發生之爭議",
"消費者依規定,以書面通知解除契約者,除當事人另有個別磋商外,企業經營者應於收到通知之次日起至少多久內,至原交付處所或約定處所取回商品?(A)40日(B)15日(C)20日(D)30日",
"依消費者保護法規定,下列關於中央主管機關之敘述何者錯誤?(A)消費者保護法所稱主管機關,在中央為目的事業主管機關(B)行政院得選擇特定行業,公告定型化契約之審閱期間(C)得選擇特定行業,擬訂定型化契約不得記載事項,報請行政院核定後公告(D)得選擇特定行業,擬訂定型化契約應記載事項,報請行政院核定後公告",
"依消費者保護法規定,未經消費者要約而對之郵寄之商品,下列相關敘述何者正確?(A)消費者須負保管義務(B)消費者得定相當期限通知商品之寄送人取回(C)商品之寄送人得請求消費者支付必要費用(D)消費者無法通知商品之寄送人時,視為成交",
"消費者保護團體對於同一之原因事件,致使眾多消費者受害時,至少應得受讓多少以上消費者損害賠償請求權後,以自己名義,提起訴訟?(A)30人(B)20人(C)15人(D)10人",
"依消費者保護法規定,下列何者為中央主管機關依契約之性質及目的,於擬訂定型化契約之不得記載事項?(A)違反契約之法律效果(B)企業經營者保留契約內容或期限之變更權(C)契約之重要權利義務事項(D)預付型交易之履約擔保",
"依消費者保護法規定,下列何者於其商品於流通進入市場,符合當時科技水準合理期待的安全性即可,不須就其主張之事實負舉證責任?(A)商品配送者(B)改裝商品者(C)變更服務內容者(D)分裝商品者",
"依消費者保護法規定,下列何者非屬消費爭議時,消費者申訴之對象?(A)企業經營者(B)消費者服務中心(C)消費者保護團體(D)鄉鎮區公所",
"依消費者保護法規定,企業經營者對消費者從事與信用有關之交易,下列有關敘述何者錯誤?(A)總費用之範圍,由目的事業主管機關定之(B)總費用之年百分率計算方式,由企業經營者定之(C)應於廣告上明示應付所有總費用之年百分率(D)總費用之範圍及年百分率計算方式,由各目的事業主管機關定之",
"當事人對於消費爭議調解委員會依職權提出解決事件之方案,得於送達後幾日之不變期間內提出異議,未於該期間內提出異議者,視為已依該方案成立調解?(A)20日(B)30日(C)15日(D)10日",
"依消費者保護法規定,企業經營者未經邀約而與消費者在其住居所、工作場所、公共場所或其他場所所訂立之契約。此係指下列何者?(A)分期付款交易(B)現物交易(C)訪問交易(D)寄賣",
"依消費者保護法規定,有關定型化契約條款之表示方式,下列何者錯誤?(A)限定以中央主管機關所提出之函釋(B)以張貼方式表示(C)以放映字幕方式表示(D)以網際網路方式表示",
"消費者保護團體對於同一之原因事件,致使眾多消費者受害時,至少應得受讓多少名以上消費者損害賠償請求權後,以自己名義,提起訴訟?(A)30人(B)15人(C)20人(D)10人",
"下列何者非屬政府為達成制定消費者保護法之目的,應實施之措施?(A)防止商品或服務損害消費者之生命、身體、健康、財產或其他權益(B)維護商品或服務之品質與安全衛生(C)促進事業商譽之維護及商品價格之保障(D)確保商品或服務之標示,符合法令規定",
"依消費者保護法規定,企業經營者以定型化契約條款使消費者拋棄消費者審閱之合理期間,其條款之效力如何?(A)得撤銷(B)有效(C)無效(D)效力未定",
"依消費者保護法規定,有關預售屋買賣定型化契約條款如有疑義時,下列敘述何者正確?(A)應為有利於不動產仲介業者之解釋(B)應為有利於不動產代銷業者解釋(C)應為有利於消費者之解釋(D)應為有利於不動產經紀人員解釋",
"依消費者保護法規定,企業經營者對第三人之損害賠償責任,下列敘述何者正確?(A)得預先約定限制(B)不得預先約定限制或免除(C)得預先約定免除(D)得預先約定減輕",
"依消費者保護法所提之訴訟,因企業經營者故意所致之損害,消費者得請求損害額至多幾倍以下之懲罰性賠償金?(A)3倍(B)2倍(C)5倍(D)1倍",
"依消費者保護法規定,下列何者就商品或服務所生之損害,與設計、生產、製造商品或提供服務之企業經營者連帶負賠償責任?(A)消費者(B)目的事業主管機關(C)行政院消費者保護處(D)從事經銷之企業經營者",
"依消費者保護法規定,下列有關個別磋商條款之敘述,何者正確?(A)契約當事人個別磋商而合意之契約條款(B)經營者逕自以放映字幕、張貼方法表示之契約條款(C)經營者逕自以書面表示之契約條款(D)經營者與多數消費者訂立契約之用,提出預先擬定之契約條款",
"企業經營者於消費者收受商品時,未依規定提供消費者解除契約相關資訊者,自收受商品後7日期間起算,至少逾多久者,解除權消滅?(A)2個月(B)3個月(C)4個月(D)1個月",
"依消費者保護法規定,定型化契約中之條款違反平等互惠原則者,該契約條款之效力如何?(A)效力未定(B)無效(C)有效(D)得撤銷"
]
answers = []


In [42]:
for query in questions:
    answers.append(rag_chain.invoke(query))

In [45]:
data = {
    "answer": answers
}

In [48]:
print(data)

{'answer': ['正確答案是 **(A) 公告不得記載之事項,經記載於定型化契約,仍構成契約之內容**\n\n說明：\n\n* 當中央主管機關針對特定行業的定型化契約公告了應記載或不得記載的事項時，這些規定具有法律效力。\n* 如果企業經營者在定型化契約中記載了公告明確禁止的條款，該條款將 **無效**，不會構成契約內容。\n\n其他選項皆為正確敘述：\n\n* (B) 違反公告之定型化契約,其定型化契約條款無效: 正確，公告具有法律效力，違反公告的條款無效。\n* (C) 公告應記載之事項,雖未記載於定型化契約,仍構成契約之內容:  正確，公告應記載的事項被視為契約必要條件，即使未明確寫入，也仍然有效。\n* (D) 企業經營者使用定型化契約者,主管機關得隨時派員查核: 正確，主管機關有權力監督企業的定型化契約是否符合規定。 \n\n\n\n\n\n', '正确答案是 **(D) 对于有事实足认有危害消费者安全与健康之虞的商品，应即回收**。\n\n以下是原因：\n\n* **(A) 错误**:  对于有危害消费者财产之虞的商品，企业经营者不仅要进行明显的警告标示，还需要采取其他措施来确保消费者安全，例如提供更详细的产品说明、使用指南等。\n* **(B) 错误**: 企业经营者不能预先约定期定免除对第三人的损害赔偿责任。法律赋予消费者一定的权利，企业经营者无法通过合同条款完全剥夺这些权利。\n* **(C) 错误**:  虽然企业经营者可以与消费者签署协议限制其部分损害赔偿责任，但对于因产品缺陷或疏忽导致的重大损害，例如人身伤害，这种限制通常无效。\n\n**消费者保护法强调企业的责任**, 尤其是在涉及消费者安全和健康方面。当商品存在危害消费者安全和健康的风险时，企业经营者有义务立即采取措施来消除风险，包括召回产品。\n\n\n\n', '正確答案是 **(B) 其條款不構成契約之內容**\n\n消費者保護法第 10 條規定，企業經營者不得以定型化契約條款使消費者拋棄消費者審閱之合理期間。 \n\n如果企業經營者違反此規定，則該條款將被視為無效，**不構成契約之內容**。\n\n這項規定旨在保護消費者的權益，確保他們有足夠的時間審閱和理解契約內容，而不是被不合理的條款限制。\n\n\n\n', '正確答案是 **(D) 以確保公平競爭、促進經濟之安定與

In [49]:
import pandas as pd
df = pd.DataFrame(answers, columns=["Answer"])

# 查看前 50 筆資料
first_50_answers = df.head(50)
print(first_50_answers)

                                               Answer
0   正確答案是 **(A) 公告不得記載之事項,經記載於定型化契約,仍構成契約之內容**\n\n...
1   正确答案是 **(D) 对于有事实足认有危害消费者安全与健康之虞的商品，应即回收**。\n\...
2   正確答案是 **(B) 其條款不構成契約之內容**\n\n消費者保護法第 10 條規定，企業...
3   正確答案是 **(D) 以確保公平競爭、促進經濟之安定與繁榮為宗旨**。\n\n消費者保護團...
4   正确答案是 **(C)**。\n\n以下是解释：\n\n* **消费者保护法** 对于进口商...
5   正確答案是 **(B) 製造商之資本額**。\n\n消費者保護法對於書面保證書的內容規定主要...
6   正確答案是 **(C)**。\n\n以下是原因：\n\n* 消費者保護團體在進行檢驗後，應公...
7   正確答案是 **(D) 指消費者與企業經營者間就商品或服務所發生之法律關係**\n\n消費者...
8   正確答案是 **(D) 定型化契約中之定型化契約條款，一部無效，該契約全部無效**\n\n以...
9   答案是 **(A) 企業經營者**。\n\n根據消費者保護法，提供商品或服務的企業經營者有責...
10  正確答案是 **(B) 以定型化契約條款使消費者拋棄合理審閱全部條款期間之權利者,無效**。...
11  正确答案是 **(A)负通知之义务**。\n\n以下是解释：\n\n* 消费者保护法保障消费...
12  正確答案是 **(C) 產品責任險之金額**。\n\n消費者保護法規定，企業在通訊交易中應提...
13  正确答案是 **(C)**。\n\n以下是解释：\n\n* 消费者保护法旨在保护消费者的权益...
14  正確答案是 **(C) 保障期間及其起算方法**。\n\n以下是消費者保護法對於分期付款買賣...
15  正確答案是 **(D) 約定須在3日內退回商品**。\n\n以下是原因：\n\n* 消費者保...
16  根據消費者保護法，消費者於收受商品後在七日內，可以**發出書面**的撤銷通知解除契約。\n\...
17  The answer is **(C) 企业经营

In [51]:
df = pd.DataFrame(answers, columns=["Answer"])

# 每次印 10 筆，分 5 次
batch_size = 10
for i in range(5):
    start = i * batch_size
    end = start + batch_size
    print(f"第 {i+1} 批：")
    print(df[start:end])
    print("\n" + "=" * 20 + "\n")  # 分隔批次

第 1 批：
                                              Answer
0  正確答案是 **(A) 公告不得記載之事項,經記載於定型化契約,仍構成契約之內容**\n\n...
1  正确答案是 **(D) 对于有事实足认有危害消费者安全与健康之虞的商品，应即回收**。\n\...
2  正確答案是 **(B) 其條款不構成契約之內容**\n\n消費者保護法第 10 條規定，企業...
3  正確答案是 **(D) 以確保公平競爭、促進經濟之安定與繁榮為宗旨**。\n\n消費者保護團...
4  正确答案是 **(C)**。\n\n以下是解释：\n\n* **消费者保护法** 对于进口商...
5  正確答案是 **(B) 製造商之資本額**。\n\n消費者保護法對於書面保證書的內容規定主要...
6  正確答案是 **(C)**。\n\n以下是原因：\n\n* 消費者保護團體在進行檢驗後，應公...
7  正確答案是 **(D) 指消費者與企業經營者間就商品或服務所發生之法律關係**\n\n消費者...
8  正確答案是 **(D) 定型化契約中之定型化契約條款，一部無效，該契約全部無效**\n\n以...
9  答案是 **(A) 企業經營者**。\n\n根據消費者保護法，提供商品或服務的企業經營者有責...


第 2 批：
                                               Answer
10  正確答案是 **(B) 以定型化契約條款使消費者拋棄合理審閱全部條款期間之權利者,無效**。...
11  正确答案是 **(A)负通知之义务**。\n\n以下是解释：\n\n* 消费者保护法保障消费...
12  正確答案是 **(C) 產品責任險之金額**。\n\n消費者保護法規定，企業在通訊交易中應提...
13  正确答案是 **(C)**。\n\n以下是解释：\n\n* 消费者保护法旨在保护消费者的权益...
14  正確答案是 **(C) 保障期間及其起算方法**。\n\n以下是消費者保護法對於分期付款買賣...
15  正確答案是 **(D) 約定須在3日內退回商品**。\n\n以下是原因：\n\n* 消費者保...
16  根據消費者保護法，消費者於收受商品後在